# 순서 DP

입력의 순서를 유지하면서 부분 구조를 찾는 DP 유형이다. 주로 수열 or 문자열에서 정의된 부분 구조 탐색 문제에서 활용된다.

- LIS (부분 수열 중 가장 긴 수열 탐색)
- LCS (두 문자열에서 공통된 최대 부분 수열 탐색)

In [1]:
# LIS
def LIS(arr):
  size = len(arr)

  # 각 위치의 LIS 길이 초기화 (자기 자신 포함 최소 길이 1)
  LIS = [1 for _ in range(size)]
  # i 번째 원소의 LIS 경로상 바로 이전 인덱스 저장
  prev = [-1 for _ in range(size)]

  # DP : 이전 원소들을 모두 확인하며 LIS 길이 갱신
  for base in range(size):
    for curr in range(base):
      # arr[curr] < arr[base] : 증가 조건 (증가 수열에 포함 가능)
      # LIS[base] < LIS[curr]+1 : 최적화 조건 (더 긴 증가 수열을 찾은 경우)
      if arr[curr] < arr[base] and LIS[base] <= LIS[curr]:
        LIS[base] = LIS[curr] + 1
        prev[base] = curr
  
  # 최장 LIS 길이 및 끝 인덱스
  max_length = max(LIS)
  idx = LIS.index(max_length)

  # prev 역추적으로 경로 복원
  path = []
  while idx != -1:
    path.append(arr[idx])
    idx = prev[idx]
  path.reverse()

  return max_length, path

max_length, path = LIS([10, 20, 10, 30, 20, 50])
print(f'LIS length : {max_length}')
print(path)

LIS length : 4
[10, 20, 30, 50]


In [2]:
# LIS를 길이 중심으로 관리하되, 같은 길이에서는 더 작은 값을 유지
from bisect import bisect_left

def LIS(arr):
  size = len(arr)

  # 길이 i+1 증가 수열 중, 마지막 인덱스 저장
  LIS = []
  # LIS[i]의 바로 이전 인덱스 저장
  prev = [-1 for _ in range(size)]

  for pos in range(size):
    val = arr[pos]

    # 현재 값의 LIS에서 위치 탐색 (이진 탐색) 
    insert_pos = bisect_left([arr[i] for i in LIS], val)

    # 기존 LIS보다 크다면 LIS 확장, 아니면 기존 LIS 위치에 삽입 (작은 값으로 변경)
    if insert_pos == len(LIS):
      LIS.append(pos)
    else:
      LIS[insert_pos] = pos

    # LIS의 길이가 1을 넘은 경우만 경로 복원용 prev 연산 수행
    if insert_pos > 0:
      prev[pos] = LIS[insert_pos - 1]

  max_length = max(LIS)
  
  # prev 역추적으로 경로 복원
  idx = LIS[-1]
  path = []
  while idx != -1:
    path.append(arr[idx])
    idx = prev[idx]
  path.reverse()

  return max_length, path

max_length, path = LIS([10, 20, 10, 30, 20, 50])
print(f'LIS length(+Binary Search) : {max_length}')
print(path)

LIS length(+Binary Search) : 5
[10, 20, 30, 50]


In [3]:
# LCS

# 방향 상수 (↖, ↑, ←)
DIAGONAL, UP, LEFT = 0, 1, 2

def LCS(s1, s2):
  size1, size2 = len(s1), len(s2)

  # LCS[r][c] : s1의 처음 i개 문자와 s2d의 처음 j개 문자로 만든 LCS 길이
  LCS = [[0 for _ in range(size2+1)] for _ in range(size1+1)]
  # from_dir[r][c] : 해당 위치가 어떤 선택에서 왔는지 기록 : 
  # 0 = DIAGONAL ↖ | s1[r-1] == s2[c-1]인 경우 (문자가 같아 증가)
  # 1 = UP       ↑ | s1[r-1]에서만 LCS를 가져온 경우 (s2 문자 무시)
  # 2 = LEFT     ← | s2[c-1]에서만 LCS를 가져온 경우 (s1 문자 무시)
  from_dir = [[-1 for _ in range(size2+1)] for _ in range(size1+1)]

  # DP : 각 문자 쌍을 비교하여 LCS 길이 갱신
  for r in range(size1):
    for c in range(size2):
      if s1[r] == s2[c]: # 두 문자열 모두에서 문자를 사용하므로, LCS[r][c] + 1
        LCS[r+1][c+1] = LCS[r][c] + 1
        from_dir[r+1][c+1] = DIAGONAL
      else:              # 둘 중 긴 LCS 경로를 선택하고, 해당 방향을 기록
        if LCS[r][c+1] >= LCS[r+1][c]:
          LCS[r+1][c+1] = LCS[r][c+1]
          from_dir[r+1][c+1] = UP
        else:
          LCS[r+1][c+1] = LCS[r+1][c]
          from_dir[r+1][c+1] = LEFT


  # 경로 복원
  r, c = size1, size2
  path = []
  while r > 0 and c > 0:
    direction = from_dir[r][c]
    if direction == DIAGONAL:
      path.append(s1[r-1])
      r -= 1
      c -= 1
    elif direction == UP:
      r -= 1
    elif direction == LEFT:
      c -= 1

  path.reverse()

  return LCS[size1][size2], path

max_length, path = LCS("ACAYKP", "CAPCAK")
print(f'LCS length : {max_length}')
print(path)

LCS length : 4
['A', 'C', 'A', 'K']
